#  Gated recurrent unit (GRU) RNNs

This chapter requires some exposition. The GRU updates are fully implemented and the code appears to work properly.

In [1]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
import numpy as np
mx.random.seed(1)
#ctx = mx.gpu(0)
ctx = mx.cpu()

## Dataset: "The Time Machine" 

In [2]:
with open("../data/nlp/timemachine.txt") as f:
    time_machine = f.read()
time_machine = time_machine[:-38083]

## Numerical representations of characters

In [3]:
character_list = list(set(time_machine))
vocab_size = len(character_list)
character_dict = {}
for e, char in enumerate(character_list):
    character_dict[char] = e
time_numerical = [character_dict[char] for char in time_machine]

## One-hot representations

In [4]:
def one_hots(numerical_list, vocab_size=vocab_size):
    result = nd.zeros((len(numerical_list), vocab_size), ctx=ctx)
    for i, idx in enumerate(numerical_list):
        result[i, idx] = 1.0
    return result

In [5]:
def textify(embedding):
    result = ""
    indices = nd.argmax(embedding, axis=1).asnumpy()
    for idx in indices:
        result += character_list[int(idx)]
    return result

## Preparing the data for training

In [6]:
batch_size = 32
seq_length = 64
# -1 here so we have enough characters for labels later
num_samples = (len(time_numerical) - 1) // seq_length
dataset = one_hots(time_numerical[:seq_length*num_samples]).reshape((num_samples, seq_length, vocab_size))
num_batches = len(dataset) // batch_size
train_data = dataset[:num_batches*batch_size].reshape((num_batches, batch_size, seq_length, vocab_size))
# swap batch_size and seq_length axis to make later access easier
train_data = nd.swapaxes(train_data, 1, 2)

## Preparing our labels

In [7]:
labels = one_hots(time_numerical[1:seq_length*num_samples+1])
train_label = labels.reshape((num_batches, batch_size, seq_length, vocab_size))
train_label = nd.swapaxes(train_label, 1, 2)

## Gated recurrent units (GRU) RNNs

Similar to LSTM blocks, the GRU also has mechanisms to enable "memorizing" information for an extended number of time steps. However, it does so in a more expedient way:

* We no longer keep a separate memory cell $c_t$. Instead, $h_{t-1}$ is added to a "new content" version of itself to give $h_{t}$.
* The "new content" version is given by $g_t = \text{tanh}(X_t W_{xh} + (r_t \odot h_{t-1})W_{hh} + b_h ),$ and is analogous to $g_t$ in the LSTM tutorial.
* Here, there is a reset gate $r_t$ which moderates the impact of $h_{t-1}$ on the "new content" version.
* The input gate $i_t$ and forget gate $f_t$ are replaced by an single update gate $z_t$, which weighs the old and new content via $z_t$ and $(1 - z_t)$ respectively.
* There is no output gate $o_t$; the weighted sum is what becomes $h_t$.

We use the GRU block with the following transformations that map inputs to outputs across blocks at consecutive layers and consecutive time steps: $\newcommand{\xb}{\mathbf{x}} \newcommand{\RR}{\mathbb{R}}$

$$z_t = \sigma(X_t W_{xz} + h_{t-1} W_{hz} + b_z),$$

$$r_t = \sigma(X_t W_{xr} + h_{t-1} W_{hr} + b_r),$$

$$g_t = \text{tanh}(X_t W_{xh} + (r_t \odot h_{t-1})W_{hh} + b_h ),$$

$$h_t = z_t \odot h_{t-1} + (1-z_t) \odot g_t,$$

where $\sigma$ and $\text{tanh}$ are as before in the LSTM case. 

Empirically, GRUs have similar performance to LSTMs, while requiring less parameters and forgoing an internal time state. Intuitively, GRUs have enough gates/state for long-term retention, but not too much, so that training and convergence remain fast and convex. See the work of Chung et al. [2014] (https://arxiv.org/abs/1412.3555).

## Allocate parameters

In [8]:
num_inputs = vocab_size
num_hidden = 256
num_outputs = vocab_size

########################
#  Weights connecting the inputs to the hidden layer
########################
Wxz = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxr = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxh = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01

########################
#  Recurrent weights connecting the hidden layer across time steps
########################
Whz = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whr = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whh = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01

########################
#  Bias vector for hidden layer
########################
bz = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
br = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bh = nd.random_normal(shape=num_hidden, ctx=ctx) * .01

########################
# Weights to the output nodes
########################
Why = nd.random_normal(shape=(num_hidden,num_outputs), ctx=ctx) * .01
by = nd.random_normal(shape=num_outputs, ctx=ctx) * .01

## Attach the gradients

In [9]:
params = [Wxz, Wxr, Wxh, Whz, Whr, Whh, bz, br, bh, Why, by]

for param in params:
    param.attach_grad()

## Softmax Activation

In [10]:
def softmax(y_linear, temperature=1.0):
    lin = (y_linear-nd.max(y_linear)) / temperature
    exp = nd.exp(lin)
    partition = nd.sum(exp, axis=0, exclude=True).reshape((-1,1))
    return exp / partition

## Define the model

In [11]:
def gru_rnn(inputs, h, temperature=1.0):
    outputs = []
    for X in inputs:
        z = nd.sigmoid(nd.dot(X, Wxz) + nd.dot(h, Whz) + bz)
        r = nd.sigmoid(nd.dot(X, Wxr) + nd.dot(h, Whr) + br)
        g = nd.tanh(nd.dot(X, Wxh) + nd.dot(r * h, Whh) + bh)
        h = z * h + (1 - z) * g
        
        yhat_linear = nd.dot(h, Why) + by
        yhat = softmax(yhat_linear, temperature=temperature) 
        outputs.append(yhat)
    return (outputs, h)

## Cross-entropy loss function

In [12]:
def cross_entropy(yhat, y):
    return - nd.mean(nd.sum(y * nd.log(yhat), axis=0, exclude=True))

## Averaging the loss over the sequence

In [13]:
def average_ce_loss(outputs, labels):
    assert(len(outputs) == len(labels))
    total_loss = nd.array([0.], ctx=ctx)
    for (output, label) in zip(outputs,labels):
        total_loss = total_loss + cross_entropy(output, label)
    return total_loss / len(outputs)

## Optimizer

In [14]:
def SGD(params, lr):    
    for param in params:
        param[:] = param - lr * param.grad

## Generating text by sampling

In [15]:
def sample(prefix, num_chars, temperature=1.0):
    #####################################
    # Initialize the string that we'll return to the supplied prefix
    #####################################
    string = prefix

    #####################################
    # Prepare the prefix as a sequence of one-hots for ingestion by RNN
    #####################################
    prefix_numerical = [character_dict[char] for char in prefix]
    input_sequence = one_hots(prefix_numerical)
    
    #####################################
    # Set the initial state of the hidden representation ($h_0$) to the zero vector
    #####################################    
    h = nd.zeros(shape=(1, num_hidden), ctx=ctx)
    c = nd.zeros(shape=(1, num_hidden), ctx=ctx)

    #####################################
    # For num_chars iterations,
    #     1) feed in the current input
    #     2) sample next character from from output distribution
    #     3) add sampled character to the decoded string
    #     4) prepare the sampled character as a one_hot (to be the next input)
    #####################################    
    for i in range(num_chars):
        outputs, h = gru_rnn(input_sequence, h, temperature=temperature)
        choice = np.random.choice(vocab_size, p=outputs[-1][0].asnumpy())
        string += character_list[choice]
        input_sequence = one_hots([choice])
    return string

In [ ]:
epochs = 2000
moving_loss = 0.

learning_rate = 2.0

# state = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
for e in range(epochs):
    ############################
    # Attenuate the learning rate by a factor of 2 every 100 epochs.
    ############################
    if ((e+1) % 100 == 0):
        learning_rate = learning_rate / 2.0
    h = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
    for i in range(num_batches):
        data_one_hot = train_data[i]
        label_one_hot = train_label[i]
        with autograd.record():
            outputs, h = gru_rnn(data_one_hot, h)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = nd.mean(loss).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(loss).asscalar()
    if e%10==0:
        print("Epoch %s. Loss: %s" % (e, moving_loss)) 
        print(sample("The Time Ma", 1024, temperature=.1))
        print(sample("The Medical Man rose, came to the lamp,", 1024, temperature=.1))
            

Epoch 0. Loss: 3.6526421483244946
The Time Ma te at te an a t at t t ta a t at t aa t ta t at te t te te te a ta t t te at te a t at t t t t te te t te a to a the aa t at te a at the t te t te o at a ta te t a ta to t ta t t t ta ta t at a t at h he t a ta o t t te t te t t aa t at t at te at t a at a t t a ate t a at t at te te at t at ate t a te t at a t at a te t t t te t t t a t t at t at t t t te an t at a at th t t te o at the t at at t te at t aa t ta te t a t t at a ta tt t at t at ae t th t te t t te a te te te t a t th t t aa t te t te te a the at t at t at te t at te t at t at t a t te at te te a t t t t a te te t aa at t at t th te t at a t te t te t to aa tae t t a t the a t aa t te t t a at t ta te t at at at t t at t ate t t th at a at t ta tt a ta ao th te te te th t ate te to t t at th a t te te te t t at ta at at te t t te t at t at at at at t a t th a te t at t a t at a tt a th at a tt at t an t a te a t atte t a t te t t at at t at t te at t te t te t te te t ta t t

## Conclusions
[Placeholder]

## Next
[Simple, LSTM, and GRU RNNs with gluon](../chapter05_recurrent-neural-networks/rnns-gluon.ipynb)

For whinges or inquiries, [open an issue on  GitHub.](https://github.com/zackchase/mxnet-the-straight-dope)